In [3]:
import json
from pprint import pprint
import logging
import requests, json
from amocrm.v2 import tokens, Pipeline, Lead
import pandas as pd
from typing import Union, Optional, Tuple
import numpy as np 
import os
from datetime import datetime

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [5]:
with open('secrets.json', 'r') as f:
    secrets = json.load(f)




storage = tokens.FileTokensStorage()
tokens.default_token_manager(
    client_id=secrets['client_id'],
    client_secret=secrets['client_secret'],
    subdomain=secrets['subdomain'],
    redirect_url=secrets['redirect_url'],
    storage=storage
)
access_token = tokens.default_token_manager.get_access_token()

if os.path.exists('events.json'):
    with open('events.json', 'r', encoding='utf-8') as f:
        EVENT_DIRECTORY = json.load(f)
else:
    EVENT_DIRECTORY = {}  # или любое другое значение по умолчанию
    print("events.json не найден.")

NoToken: You need to init tokens with code by 'init' method

In [19]:

class SpecificDataProcessing:
    """
    Обработка specific_data в зависимоси от типа входящей строки
    """
    
    def __init__(self):
        """
        entity_linked_func - получить даные о клиенте/компании
        sale_field_changed_func- поулчить данные о цене 
        lead_status_func - получить данные о статусе задачи 
        pipline_func - получить даные о pipline 
        
        sale: float - цена 
        company: int - id компании 
        client: int - id клиента
        lead_status: int - id статуса задачи 
        pipline: int - id пайплайна
        """
        
        # логика обработки для разных типов specific_data
        self.entity_linked_func = lambda row: row.specific_data['after'][0]['link']['entity']['id']                
        self.sale_field_changed_func = lambda row: row.specific_data['after'][0]['sale_field_value']['sale']
        self.lead_status_func = lambda row: row.specific_data['after'][0]['lead_status']['id']
        self.pipline_func = lambda row: row.specific_data['after'][0]['lead_status']['pipeline_id'] 
        
        # начальная инициализация сквозных значений (sale, responsible_user_id, pipeline, lead_status)
        self.initial_func= lambda row: (row.specific_data['sale'], 
                                        row.specific_data['responsible_user_id'],
                                        row.specific_data['pipeline'],
                                        row.specific_data['lead_status'])
        
        # сквозные поля датасета
        self.sale: Optional[float] = np.nan
        self.company: Optional[str]= np.nan 
        self.contact: Optional[str]= np.nan
        self.lead_status: Optional[str] = np.nan
        self.pipline: Optional[int]= np.nan 
        self.responsible: Optional[int]= np.nan 

        
    
    def __call__(self, row: pd.Series) -> pd.Series:
        """  
        Обработка поля specific_data для получени сквозных показателей
        
        Аргументы:
            row: строка данных 
        Возвращает:
            pd.Series для следующих полей:
                ('client', 'company', 'sale', 'lead_status', 'pipline')
        """
        
        # если это строка инициализации задачи
        if row.type == 'initial':
            self.sale, self.responsible, self.pipline, self.lead_status = self.initial_func(row)
        
        # если установили/изменили sale
        elif row.type == 'sale_field_changed':
           self.sale =  self.sale_field_changed_func(row)
           
        # если добавили/изменили сущности: company, contact
        elif row.type=='entity_linked':
            if row.specific_data['after'][0]['link']['entity']['type']=='contact':
                 self.contact = self.entity_linked_func(row)
            elif row.specific_data['after'][0]['link']['entity']['type']=='company':
                self.company = self.entity_linked_func(row)
                
        # если изменился статус/пайплайн задачи задачи
        elif row.type=='lead_status_changed':
            self.lead_status = self.lead_status_func(row)
            self.pipline = self.pipline_func(row)

        return pd.Series([self.contact, self.company, self.sale, self.lead_status, self.pipline, self.responsible])

In [33]:
class AmoCRM:
    def __init__(self, token_manager, secrets):
        """ 
        Получение и обработка джанных AmoCRM
        
        Аргументы:
            token_manager - получение и обновление токена для доступа к данным 
            subdomain - поддомен аккаунта в autocrm 
            processor - класс для обработки сквозных значений
            _general_fields - поля основных значений (одинаковые для каждого типа записи)
        """
        self.token_manager = token_manager
        self.subdomain = secrets["subdomain"]
        self.processor = SpecificDataProcessing
        self._general_fields = ['type', 'entity_id', 'created_by', 'created_at', 'specific_data']

    def _api_call(self, endpoint, entity, entity_id):
        headers = {
            'Authorization': f'Bearer {self.token_manager.get_access_token()}',
            'Content-Type': 'application/json'
        }

        url = f'https://{self.subdomain}.amocrm.ru/api/v4/{endpoint}'

        params = {
            "filter[entity]": entity,
            "filter[entity_id]": entity_id
        }

        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response
    

    def _initial_processing(self, initial_df: pd.DataFrame)-> pd.DataFrame:
        ''' 
        Унификация инициализирующих данных
        '''

        initial_df['type']='initial',
        initial_df['entity_id']=initial_df['id']
        initial_df['specific_data'] = initial_df.apply(lambda row: {'sale': row.price, 
                                                                    'responsible_user_id': row.responsible_user_id, 
                                                                    'pipeline': row.pipeline_id, 
                                                                    'lead_status': row.status_id}, axis=1)
        
        return initial_df[self._general_fields]
        
    
    def _event_processing(self, event_df: pd.DataFrame) -> pd.DataFrame:
        '''
        Унификация данных по евентам
        '''
        
        event_df['specific_data'] = event_df.apply(
            lambda row: {'after':row.value_after, 'before':row.value_before}, axis=1)
        
        return event_df[self._general_fields]
    
    def _task_processing(self, task_df: pd.DataFrame) -> pd.DataFrame:
        '''
        Унификация данных по задачам
        '''
        
        task_df['type'] = 'task'
        task_df['specific_data'] = task_df.apply(
            lambda row: {'text': row.text, 
                         'is_completed': row.is_completed,
                         'result':row.result, 
                         'responsible_user_id': row.responsible_user_id,
                         'complete_till': row.complete_till}, axis=1)
        
        return  task_df[self._general_fields ]


    def _note_processing(self, note_df: pd.DataFrame) -> pd.DataFrame:
        '''
        Унификация данных по заметкам
        '''

        note_df['type'] = 'note' 
        note_df['specific_data'] = note_df.apply(
            lambda row: {
                'text': row.params.get('text', None),  # Use .get() and provide a default value of None
                'note_type': row.note_type,
                'responsible_user_id': row.responsible_user_id,
                'updated_at': row.updated_at
            }, 
            axis=1
        )

        return note_df[self._general_fields]


    def get_initial_data_lead(self, lead_id):
        """ 
        Получение начальных данных по задаче
        """
        
        headers = {
        'Authorization': f'Bearer {self.token_manager.get_access_token()}',
        'Content-Type': 'application/json'
        }

        url = f'https://{self.subdomain}.amocrm.ru/api/v4/leads/{lead_id}'
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            initial_df = pd.Series(response.json()).to_frame().T # 
            return  self._initial_processing(initial_df)
        
        elif response.status_code == 204:
            return pd.DataFrame()
        
        else:
            raise Exception('Error: {}'.format(response.status_code))
        

    def get_events_by_lead_id(self, lead_id):
        response = self._api_call('events', 'lead', lead_id)
        if response.status_code == 200:
            event_df_raw = pd.DataFrame(response.json()['_embedded']['events'])
            return self._event_processing(event_df_raw)
        
        elif response.status_code == 204:
            return pd.DataFrame()
        
        else:
            raise Exception('Error: {}'.format(response.status_code))


    def get_tasks_by_lead_id(self, lead_id):
        response = self._api_call('tasks', 'lead', lead_id)
        if response.status_code == 200:
            task_df_raw = pd.DataFrame(response.json()['_embedded']['tasks'])
            return self._task_processing(task_df_raw)
        
        elif response.status_code == 204:
            return pd.DataFrame()
        
        else:
            raise Exception('Error: {}'.format(response.status_code))


    def get_notes_by_lead_id(self, lead_id):
        response = self._api_call('leads/notes', 'lead', lead_id)
        if response.status_code == 200:
            note_df_raw = pd.DataFrame(response.json()['_embedded']['notes'])
            return self._note_processing(note_df_raw)
        
        elif response.status_code == 204:
            return pd.DataFrame()
        
        else:
            raise Exception('Error: {}'.format(response.status_code))
    
    # preparation all_lead_info
    def get_all_lead_info(self, lead_id):
        """
        Получение и обоработка все данных по задачам
        """
        
        processor = self.processor() # обработчик сквозных значений
        inital_df = self.get_initial_data_lead(lead_id)
        events_df = self.get_events_by_lead_id(lead_id)
        tasks_df = self.get_tasks_by_lead_id(lead_id)
        notes_df = self.get_notes_by_lead_id(lead_id)

        result = pd.concat([inital_df, events_df, tasks_df, notes_df], axis=0)\
                                                .sort_values('created_at').\
                                                reset_index(drop=True).\
                                                assign(client=None,
                                                company=None,
                                                sale=None,  
                                                lead_status=None,
                                                pipline=None,
                                                responsible=None)
        result[['client', 'company', 'sale', 'lead_status', 'pipline', 'responsible']] = result.apply(processor , axis=1)
        
        #print(result['created_at'][-3:])

      
        return result.astype({'client': pd.Int32Dtype(), 
                              'company': pd.Int32Dtype(), 
                              'sale': pd.Float32Dtype(),
                              'lead_status': pd.Int32Dtype(),
                              'pipline':pd.Int32Dtype(),
                              'responsible':pd.Int32Dtype()})
# example
#lead_id = 22993645 
#lead_id = 24651711
lead_id =37925381
crm = AmoCRM(tokens.default_token_manager, secrets)
#events_list = crm.get_all_lead_info(lead_id)

In [36]:
crm.get_all_lead_info(lead_id)

,type,entity_id,created_by,created_at,specific_data,client,company,sale,lead_status,pipline,responsible
0,initial,37925381,0,1694696449,"{'sale': 111361, 'responsible_user_id': 153122...",<NA>,<NA>,111361.0,60363386,7236182,1531225
1,lead_added,37925381,0,1694696449,"{'after': [], 'before': []}",<NA>,<NA>,111361.0,60363386,7236182,1531225
2,name_field_changed,37925381,0,1694696449,{'after': [{'name_field_value': {'name': 'Сдел...,<NA>,<NA>,111361.0,60363386,7236182,1531225
3,sale_field_changed,37925381,0,1694696449,{'after': [{'sale_field_value': {'sale': 41436...,<NA>,<NA>,41436.0,60363386,7236182,1531225
4,task,37925381,0,1695219336,"{'text': 'текст из АПИ', 'is_completed': False...",<NA>,<NA>,41436.0,60363386,7236182,1531225
5,entity_linked,37925381,10076546,1695219951,{'after': [{'link': {'entity': {'type': 'compa...,<NA>,66631755,41436.0,60363386,7236182,1531225
6,entity_linked,37925381,10076546,1695219951,{'after': [{'link': {'entity': {'type': 'conta...,67602891,66631755,41436.0,60363386,7236182,1531225
7,note,37925381,10076546,1696324182,{'text': 'Обязательно выполнить эту сделку! Эт...,67602891,66631755,41436.0,60363386,7236182,1531225
8,common_note_added,37925381,10076546,1696324182,"{'after': [{'note': {'id': 395884613}}], 'befo...",67602891,66631755,41436.0,60363386,7236182,1531225
9,task,37925381,10076546,1696335621,"{'text': 'Задача для Богдана, СРОЧНО!', 'is_co...",67602891,66631755,41436.0,60363386,7236182,1531225


In [13]:
def get_managers_list(tokens, secrets):
    """
    Получение списка менеджеров из AmoCRM
    """
    headers = {
        'Authorization': f'Bearer {tokens.default_token_manager.get_access_token()}',
        'Content-Type': 'application/json'
    }
    url = f'https://{secrets["subdomain"]}.amocrm.ru/api/v4/users'
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        users = data.get('_embedded', {}).get('users', [])
        return pd.DataFrame(users)
    else:
        raise Exception('Error: {}'.format(response.status_code))

managers_list = get_managers_list(tokens, secrets)
managers_list

,id,name,email,lang,rights,_links
0,1531225,mlenzovet@gmail.com,mlenzovet@gmail.com,ru,"{'leads': {'view': 'A', 'edit': 'A', 'add': 'A...",{'self': {'href': 'https://crmsum1.amocrm.ru/a...
1,2794642,Технический Пользователь,techuseramo@team.amocrm.com,ru,"{'leads': {'view': 'A', 'edit': 'A', 'add': 'A...",{'self': {'href': 'https://crmsum1.amocrm.ru/a...
2,10076546,bulatovbogdann@gmail.com,bulatovbogdann@gmail.com,ru,"{'leads': {'view': 'A', 'edit': 'A', 'add': 'A...",{'self': {'href': 'https://crmsum1.amocrm.ru/a...


In [9]:
def replace_user_id_with_name(data_list, managers_df):
    id_to_name = managers_df.set_index('id')['name'].to_dict()
    for item in data_list:
        if 'responsible_user_id' in item:
            user_id = item['responsible_user_id']
            if user_id in id_to_name:
                item['responsible_user_id'] = id_to_name[user_id]
    return data_list

specific_data = replace_user_id_with_name(specific_data, managers_list)
specific_data


[{'sale': 0,
  'responsible_user_id': 'Валерия Бильская',
  'pipeline': 1924486,
  'lead_status': 28971487},
 {'after': [{'link': {'entity': {'type': 'contact', 'id': 47888395}}}],
  'before': []},
 {'after': [], 'before': []},
 {'after': [{'name_field_value': {'name': 'ГАЛС'}}], 'before': []},
 {'after': [{'custom_field_value': {'field_id': 487035,
     'field_type': 4,
     'enum_id': 955727,
     'text': '16 Ждет от нас'}}],
  'before': []},
 {'text': 'много ебли в тг :)',
  'note_type': 'common',
  'responsible_user_id': 'Валерия Бильская',
  'updated_at': 1663233278},
 {'after': [{'note': {'id': 250270209}}], 'before': []},
 {'after': [{'custom_field_value': {'field_id': 487035,
     'field_type': 4,
     'enum_id': 955373,
     'text': '5 Тестируют систему'}}],
  'before': [{'custom_field_value': {'field_id': 487035,
     'field_type': 4,
     'enum_id': 955727,
     'text': '16 Ждет от нас'}}]},
 {'after': [], 'before': []},
 {'after': [{'message': {'id': 'df72796b-13ac-469e-b21

In [10]:
def get_pipelines_list(tokens, secrets):
    """
    Получение списка pipelines из AmoCRM
    """
    headers = {
        'Authorization': f'Bearer {tokens.default_token_manager.get_access_token()}',
        'Content-Type': 'application/json'
    }
    url = f'https://{secrets["subdomain"]}.amocrm.ru/api/v4/leads/pipelines'
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        pipelines = data.get('_embedded', {}).get('pipelines', [])
        return pd.DataFrame(pipelines)
    else:
        raise Exception('Error: {}'.format(response.status_code))

pipelines_list = get_pipelines_list(tokens, secrets)
pipelines_list


,id,name,sort,is_main,is_unsorted_on,is_archive,account_id,_links,_embedded
0,718057,Воронка,1,True,True,False,15895585,{'self': {'href': 'https://lightson.amocrm.ru/...,"{'statuses': [{'id': 26125648, 'name': 'Неразо..."
1,1924486,Карты офиса,2,False,True,False,15895585,{'self': {'href': 'https://lightson.amocrm.ru/...,"{'statuses': [{'id': 28971484, 'name': 'Неразо..."
2,3460750,Холодные продажи,3,False,True,False,15895585,{'self': {'href': 'https://lightson.amocrm.ru/...,"{'statuses': [{'id': 34333918, 'name': 'Неразо..."
3,3464731,Презентации,4,False,True,False,15895585,{'self': {'href': 'https://lightson.amocrm.ru/...,"{'statuses': [{'id': 34361767, 'name': 'Неразо..."
4,4196131,Поиск клиентов,5,False,True,False,15895585,{'self': {'href': 'https://lightson.amocrm.ru/...,"{'statuses': [{'id': 39438712, 'name': 'Неразо..."
5,5474143,Отели,6,False,True,False,15895585,{'self': {'href': 'https://lightson.amocrm.ru/...,"{'statuses': [{'id': 48481912, 'name': 'Неразо..."
6,6432642,LightsON,7,False,True,False,15895585,{'self': {'href': 'https://lightson.amocrm.ru/...,"{'statuses': [{'id': 54985142, 'name': 'Неразо..."


In [11]:
def replace_user_id_with_name(data_list, managers_df, pipelines_df):
    id_to_name = managers_df.set_index('id')['name'].to_dict()
    id_to_pipeline = pipelines_df.set_index('id')['name'].to_dict()
    
    for item in data_list:
        if 'responsible_user_id' in item:
            user_id = item['responsible_user_id']
            if user_id in id_to_name:
                item['responsible_user_id'] = id_to_name[user_id]
        if 'pipeline' in item:
            pipeline_id = item['pipeline']
            if pipeline_id in id_to_pipeline:
                item['pipeline'] = id_to_pipeline[pipeline_id]
    
    return data_list

specific_data = replace_user_id_with_name(specific_data, managers_list, pipelines_list)
specific_data


[{'sale': 0,
  'responsible_user_id': 'Валерия Бильская',
  'pipeline': 'Карты офиса',
  'lead_status': 28971487},
 {'after': [{'link': {'entity': {'type': 'contact', 'id': 47888395}}}],
  'before': []},
 {'after': [], 'before': []},
 {'after': [{'name_field_value': {'name': 'ГАЛС'}}], 'before': []},
 {'after': [{'custom_field_value': {'field_id': 487035,
     'field_type': 4,
     'enum_id': 955727,
     'text': '16 Ждет от нас'}}],
  'before': []},
 {'text': 'много ебли в тг :)',
  'note_type': 'common',
  'responsible_user_id': 'Валерия Бильская',
  'updated_at': 1663233278},
 {'after': [{'note': {'id': 250270209}}], 'before': []},
 {'after': [{'custom_field_value': {'field_id': 487035,
     'field_type': 4,
     'enum_id': 955373,
     'text': '5 Тестируют систему'}}],
  'before': [{'custom_field_value': {'field_id': 487035,
     'field_type': 4,
     'enum_id': 955727,
     'text': '16 Ждет от нас'}}]},
 {'after': [], 'before': []},
 {'after': [{'message': {'id': 'df72796b-13ac-46

In [12]:
def get_lead_statuses_list(tokens, secrets):
    """
    Получение списка статусов сделок из AmoCRM
    """
    headers = {
        'Authorization': f'Bearer {tokens.default_token_manager.get_access_token()}',
        'Content-Type': 'application/json'
    }
    url = f'https://{secrets["subdomain"]}.amocrm.ru/api/v4/leads/pipelines'
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        pipelines = data.get('_embedded', {}).get('pipelines', [])
        statuses = [status for pipeline in pipelines for status in pipeline['_embedded']['statuses']]
        return pd.DataFrame(statuses)
    else:
        raise Exception('Error: {}'.format(response.status_code))

lead_statuses_list = get_lead_statuses_list(tokens, secrets)
lead_statuses_list[:3]

,id,name,sort,is_editable,pipeline_id,color,type,account_id,_links
0,26125648,Неразобранное,10,False,718057,#c1c1c1,1,15895585,{'self': {'href': 'https://lightson.amocrm.ru/...
1,28973524,Бриф,20,True,718057,#99ccff,0,15895585,{'self': {'href': 'https://lightson.amocrm.ru/...
2,15995335,КП отправлено,30,True,718057,#eb93ff,0,15895585,{'self': {'href': 'https://lightson.amocrm.ru/...


In [13]:
def replace_user_id_with_name(data_list, managers_df, pipelines_df, lead_statuses_df):
    id_to_name = managers_df.set_index('id')['name'].to_dict()
    id_to_pipeline = pipelines_df.set_index('id')['name'].to_dict()
    id_to_lead_status = lead_statuses_df.set_index('id')['name'].to_dict()
    
    for item in data_list:
        # Replace 'responsible_user_id' key
        if 'responsible_user_id' in item:
            user_id = item['responsible_user_id']
            if user_id in id_to_name:
                item['responsible_user_id'] = id_to_name[user_id]
        
        # Check for nested 'responsible_user' and 'lead_status' key in 'after' and 'before' lists
        for key in ['after', 'before']:
            if key in item:
                for nested_item in item[key]:
                    if 'responsible_user' in nested_item:
                        user_id = nested_item['responsible_user']['id']
                        if user_id in id_to_name:
                            nested_item['responsible_user']['id'] = id_to_name[user_id]
                    
                    if 'lead_status' in nested_item:
                        status_id = nested_item['lead_status']['id']
                        pipeline_id = nested_item['lead_status']['pipeline_id']
                        
                        if status_id in id_to_lead_status:
                            nested_item['lead_status']['id'] = id_to_lead_status[status_id]
                        if pipeline_id in id_to_pipeline:
                            nested_item['lead_status']['pipeline_id'] = id_to_pipeline[pipeline_id]
        
        # Replace 'pipeline' key
        if 'pipeline' in item:
            pipeline_id = item['pipeline']
            if pipeline_id in id_to_pipeline:
                item['pipeline'] = id_to_pipeline[pipeline_id]
        
        # Replace 'lead_status' key
        if 'lead_status' in item:
            lead_status_id = item['lead_status']
            if lead_status_id in id_to_lead_status:
                item['lead_status'] = id_to_lead_status[lead_status_id]
    
    return data_list
specific_data = replace_user_id_with_name(specific_data, managers_list, pipelines_list, lead_statuses_list)
specific_data

[{'sale': 0,
  'responsible_user_id': 'Валерия Бильская',
  'pipeline': 'Карты офиса',
  'lead_status': 'Получить запрос'},
 {'after': [{'link': {'entity': {'type': 'contact', 'id': 47888395}}}],
  'before': []},
 {'after': [], 'before': []},
 {'after': [{'name_field_value': {'name': 'ГАЛС'}}], 'before': []},
 {'after': [{'custom_field_value': {'field_id': 487035,
     'field_type': 4,
     'enum_id': 955727,
     'text': '16 Ждет от нас'}}],
  'before': []},
 {'text': 'много ебли в тг :)',
  'note_type': 'common',
  'responsible_user_id': 'Валерия Бильская',
  'updated_at': 1663233278},
 {'after': [{'note': {'id': 250270209}}], 'before': []},
 {'after': [{'custom_field_value': {'field_id': 487035,
     'field_type': 4,
     'enum_id': 955373,
     'text': '5 Тестируют систему'}}],
  'before': [{'custom_field_value': {'field_id': 487035,
     'field_type': 4,
     'enum_id': 955727,
     'text': '16 Ждет от нас'}}]},
 {'after': [], 'before': []},
 {'after': [{'message': {'id': 'df72796

In [14]:
def get_fields_list(tokens, secrets):
    """
    Получение списка полей из AmoCRM
    """
    headers = {
        'Authorization': f'Bearer {tokens.default_token_manager.get_access_token()}',
        'Content-Type': 'application/json'
    }
    url = f'https://{secrets["subdomain"]}.amocrm.ru/api/v4/leads/custom_fields'
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        fields = data.get('_embedded', {}).get('custom_fields', [])
        return pd.DataFrame(fields)
    else:
        raise Exception('Error: {}'.format(response.status_code))

fields_list = get_fields_list(tokens, secrets)
fields_list[:5]


,id,name,type,account_id,code,sort,is_api_only,enums,group_id,required_statuses,is_deletable,is_predefined,entity_type,tracking_callback,remind,triggers,currency,hidden_statuses,chained_lists,_links
0,485429,TRANID,textarea,15895585,,519,True,None,None,[],True,False,leads,None,None,[],None,[],None,{'self': {'href': 'https://lightson.amocrm.ru/...
1,485431,FORMNAME,textarea,15895585,,518,True,None,None,[],True,False,leads,None,None,[],None,[],None,{'self': {'href': 'https://lightson.amocrm.ru/...
2,485433,FORMID,textarea,15895585,,517,True,None,None,[],True,False,leads,None,None,[],None,[],None,{'self': {'href': 'https://lightson.amocrm.ru/...
3,485435,COOKIES,textarea,15895585,,516,True,None,None,[],True,False,leads,None,None,[],None,[],None,{'self': {'href': 'https://lightson.amocrm.ru/...
4,485437,REFERER,textarea,15895585,,513,True,None,None,[],True,False,leads,None,None,[],None,[],None,{'self': {'href': 'https://lightson.amocrm.ru/...


In [15]:
def replace_user_id_with_name(data_list, managers_df, pipelines_df, lead_statuses_df, fields_list):
    id_to_name = managers_df.set_index('id')['name'].to_dict()
    id_to_pipeline = pipelines_df.set_index('id')['name'].to_dict()
    id_to_lead_status = lead_statuses_df.set_index('id')['name'].to_dict()
    field_id_to_name = fields_list.set_index('id')['name'].to_dict()
    
    for item in data_list:
        # Replace 'responsible_user_id' key
        if 'responsible_user_id' in item:
            user_id = item['responsible_user_id']
            if user_id in id_to_name:
                item['responsible_user_id'] = id_to_name[user_id]
        
        # Check for nested 'responsible_user', 'lead_status', and 'custom_field_value' keys in 'after' and 'before' lists
        for key in ['after', 'before']:
            if key in item:
                for nested_item in item[key]:
                    if 'responsible_user' in nested_item:
                        user_id = nested_item['responsible_user']['id']
                        if user_id in id_to_name:
                            nested_item['responsible_user']['id'] = id_to_name[user_id]
                    
                    if 'lead_status' in nested_item:
                        status_id = nested_item['lead_status']['id']
                        pipeline_id = nested_item['lead_status']['pipeline_id']
                        
                        if status_id in id_to_lead_status:
                            nested_item['lead_status']['id'] = id_to_lead_status[status_id]
                        if pipeline_id in id_to_pipeline:
                            nested_item['lead_status']['pipeline_id'] = id_to_pipeline[pipeline_id]
                    
                    if 'custom_field_value' in nested_item:
                        field_id = nested_item['custom_field_value']['field_id']
                        if field_id in field_id_to_name:
                            nested_item['custom_field_value']['field_id'] = field_id_to_name[field_id]
        
        # Replace 'pipeline' key
        if 'pipeline' in item:
            pipeline_id = item['pipeline']
            if pipeline_id in id_to_pipeline:
                item['pipeline'] = id_to_pipeline[pipeline_id]
        
        # Replace 'lead_status' key
        if 'lead_status' in item:
            lead_status_id = item['lead_status']
            if lead_status_id in id_to_lead_status:
                item['lead_status'] = id_to_lead_status[lead_status_id]
    
    return data_list
specific_data = replace_user_id_with_name(specific_data, managers_list, pipelines_list, lead_statuses_list, fields_list)
specific_data

[{'sale': 0,
  'responsible_user_id': 'Валерия Бильская',
  'pipeline': 'Карты офиса',
  'lead_status': 'Получить запрос'},
 {'after': [{'link': {'entity': {'type': 'contact', 'id': 47888395}}}],
  'before': []},
 {'after': [], 'before': []},
 {'after': [{'name_field_value': {'name': 'ГАЛС'}}], 'before': []},
 {'after': [{'custom_field_value': {'field_id': 'Почему не купили',
     'field_type': 4,
     'enum_id': 955727,
     'text': '16 Ждет от нас'}}],
  'before': []},
 {'text': 'много ебли в тг :)',
  'note_type': 'common',
  'responsible_user_id': 'Валерия Бильская',
  'updated_at': 1663233278},
 {'after': [{'note': {'id': 250270209}}], 'before': []},
 {'after': [{'custom_field_value': {'field_id': 'Почему не купили',
     'field_type': 4,
     'enum_id': 955373,
     'text': '5 Тестируют систему'}}],
  'before': [{'custom_field_value': {'field_id': 'Почему не купили',
     'field_type': 4,
     'enum_id': 955727,
     'text': '16 Ждет от нас'}}]},
 {'after': [], 'before': []},
 {'

In [16]:
specific_data[2]

{'after': [], 'before': []}

In [17]:
def get_contacts_list(tokens, secrets):
    """
    Получение списка контактов из AmoCRM
    """
    headers = {
        'Authorization': f'Bearer {tokens.default_token_manager.get_access_token()}',
        'Content-Type': 'application/json'
    }
    url = f'https://{secrets["subdomain"]}.amocrm.ru/api/v4/contacts'
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        contacts = data.get('_embedded', {}).get('contacts', [])
        return pd.DataFrame(contacts)
    else:
        raise Exception('Error: {}'.format(response.status_code))

contacts_list = get_contacts_list(tokens, secrets)
contacts_list[:2]


,id,name,first_name,last_name,responsible_user_id,group_id,created_by,updated_by,created_at,updated_at,closest_task_at,is_deleted,is_unsorted,custom_fields_values,account_id,_links,_embedded
0,31376941,Екатерина Волкова,None,None,2269081,0,2269081,2269081,1534950544,1534950544,NaN,False,False,"[{'field_id': 403487, 'field_name': 'Email', '...",15895585,{'self': {'href': 'https://lightson.amocrm.ru/...,"{'tags': [], 'companies': [{'id': 31270789, '_..."
1,31516679,Влада Белявская,None,None,2269081,0,2269081,1654342,1535991702,1549359190,NaN,False,False,"[{'field_id': 403485, 'field_name': 'Телефон',...",15895585,{'self': {'href': 'https://lightson.amocrm.ru/...,"{'tags': [], 'companies': [{'id': 29605627, '_..."


In [18]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
print(contacts_list[:2])

         id               name first_name last_name  responsible_user_id  \
0  31376941  Екатерина Волкова  None       None      2269081               
1  31516679  Влада Белявская    None       None      2269081               

   group_id  created_by  updated_by  created_at  updated_at  closest_task_at  \
0  0         2269081     2269081     1534950544  1534950544 NaN                
1  0         2269081     1654342     1535991702  1549359190 NaN                

   is_deleted  is_unsorted  \
0  False       False         
1  False       False         

                                                                                                                                                                                                                                                                                                                                                        custom_fields_values  \
0  [{'field_id': 403487, 'field_name': 'Email', 'field_code': 'EMAIL', 

C:\Users\ADMINI~1\AppData\Local\Temp\2/ipykernel_11980/2177887734.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [19]:
contacts_list.shape

(250, 17)

In [20]:
import requests
import pandas as pd

def get_contact_by_id(tokens, secrets, contact_id):
    """
    Получение контакта по ID из AmoCRM
    """
    headers = {
        'Authorization': f'Bearer {tokens.default_token_manager.get_access_token()}',
        'Content-Type': 'application/json'
    }
    url = f'https://{secrets["subdomain"]}.amocrm.ru/api/v4/contacts/{contact_id}'
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        raise Exception('Error: {}'.format(response.status_code))

contact_id = 31376941
contact = get_contact_by_id(tokens, secrets, contact_id)
print(contact)


{'id': 31376941, 'name': 'Екатерина Волкова', 'first_name': None, 'last_name': None, 'responsible_user_id': 2269081, 'group_id': 0, 'created_by': 2269081, 'updated_by': 2269081, 'created_at': 1534950544, 'updated_at': 1534950544, 'closest_task_at': None, 'is_deleted': False, 'is_unsorted': False, 'custom_fields_values': [{'field_id': 403487, 'field_name': 'Email', 'field_code': 'EMAIL', 'field_type': 'multitext', 'values': [{'value': 'Ekaterina.Volkova@ipaper.com', 'enum_id': 850537, 'enum_code': 'WORK'}]}, {'field_id': 403483, 'field_name': 'Должность', 'field_code': 'POSITION', 'field_type': 'text', 'values': [{'value': 'Младший специалист Департамента снабжения'}]}], 'account_id': 15895585, '_links': {'self': {'href': 'https://lightson.amocrm.ru/api/v4/contacts/31376941?page=1&limit=250'}}, '_embedded': {'tags': [], 'companies': [{'id': 31270789, '_links': {'self': {'href': 'https://lightson.amocrm.ru/api/v4/companies/31270789?page=1&limit=250'}}}]}}


In [21]:
def add_contact_info_using_function(dict_list, tokens, secrets, get_contact_func):
    for item in dict_list:
        # Проверяем наличие информации о контакте
        if 'after' in item:
            for inner_item in item['after']:
                if 'link' in inner_item:
                    contact_info = inner_item['link']['entity']
                    if contact_info and contact_info['type'] == 'contact':
                        contact_id = contact_info['id']
                        # Используем функцию get_contact_by_id для получения данных о контакте
                        contact_row = get_contact_func(tokens, secrets, contact_id)
                        # Добавляем информацию о контакте в список словарей
                        item['contact_info'] = contact_row
    return dict_list
specific_data = add_contact_info_using_function(specific_data, tokens, secrets, get_contact_by_id)
specific_data

[{'sale': 0,
  'responsible_user_id': 'Валерия Бильская',
  'pipeline': 'Карты офиса',
  'lead_status': 'Получить запрос'},
 {'after': [{'link': {'entity': {'type': 'contact', 'id': 47888395}}}],
  'before': [],
  'contact_info': {'id': 47888395,
   'name': 'Юрий Федосеев',
   'first_name': 'Юрий',
   'last_name': 'Федосеев',
   'responsible_user_id': 2923960,
   'group_id': 415390,
   'created_by': 2923960,
   'updated_by': 2923960,
   'created_at': 1662063610,
   'updated_at': 1663076135,
   'closest_task_at': None,
   'is_deleted': False,
   'is_unsorted': False,
   'custom_fields_values': [{'field_id': 403487,
     'field_name': 'Email',
     'field_code': 'EMAIL',
     'field_type': 'multitext',
     'values': [{'value': 'fedoseevya@hals-development.ru',
       'enum_id': 850537,
       'enum_code': 'WORK'}]}],
   'account_id': 15895585,
   '_links': {'self': {'href': 'https://lightson.amocrm.ru/api/v4/contacts/47888395?page=1&limit=250'}},
   '_embedded': {'tags': [], 'companies'

In [22]:
def convert_dates_recursive(data_item):
    """
    Recursively checks for unix timestamps and converts them to the format '%Y-%m-%d %H:%M'.
    """
    if isinstance(data_item, list):
        return [convert_dates_recursive(sub_item) for sub_item in data_item]
    
    elif isinstance(data_item, dict):
        for key, value in data_item.items():
            if key in ["updated_at", "complete_till", "created_at"]:
                if isinstance(value, int):
                    data_item[key] = datetime.utcfromtimestamp(value).strftime('%Y-%m-%d %H:%M')
                elif isinstance(value, str) and len(value) == 10 and value.isnumeric():  # extra check if timestamp is string
                    data_item[key] = datetime.utcfromtimestamp(int(value)).strftime('%Y-%m-%d %H:%M')
            else:
                data_item[key] = convert_dates_recursive(value)
        return data_item
    
    else:
        return data_item


specific_data = convert_dates_recursive(specific_data)
specific_data

[{'sale': 0,
  'responsible_user_id': 'Валерия Бильская',
  'pipeline': 'Карты офиса',
  'lead_status': 'Получить запрос'},
 {'after': [{'link': {'entity': {'type': 'contact', 'id': 47888395}}}],
  'before': [],
  'contact_info': {'id': 47888395,
   'name': 'Юрий Федосеев',
   'first_name': 'Юрий',
   'last_name': 'Федосеев',
   'responsible_user_id': 2923960,
   'group_id': 415390,
   'created_by': 2923960,
   'updated_by': 2923960,
   'created_at': '2022-09-01 20:20',
   'updated_at': '2022-09-13 13:35',
   'closest_task_at': None,
   'is_deleted': False,
   'is_unsorted': False,
   'custom_fields_values': [{'field_id': 403487,
     'field_name': 'Email',
     'field_code': 'EMAIL',
     'field_type': 'multitext',
     'values': [{'value': 'fedoseevya@hals-development.ru',
       'enum_id': 850537,
       'enum_code': 'WORK'}]}],
   'account_id': 15895585,
   '_links': {'self': {'href': 'https://lightson.amocrm.ru/api/v4/contacts/47888395?page=1&limit=250'}},
   '_embedded': {'tags':

In [23]:
def remove_last_name(name):
    # This is a placeholder function. The actual function might have more logic.
    name = str(name)
    return name.split()[0] if " " in name else name

# Original function
def replace_texts_in_data(data_list):
    for item in data_list:
        for key in ['after', 'before']:
            if key in item:
                for nested_item in item[key]:
                    for sub_key, sub_value in nested_item.items():
                        if isinstance(sub_value, dict) and 'name' in sub_value and 'карта-офиса.рф' in sub_value['name']:
                            sub_value['name'] = sub_value['name'].replace('карта-офиса.рф', 'источник 1')
                        if isinstance(sub_value, dict) and 'text' in sub_value and 'xn----7sbab3bi5annfz.xn--p1ai' in sub_value['text']:
                            sub_value['text'] = sub_value['text'].replace('xn----7sbab3bi5annfz.xn--p1ai', 'наш прекрасный сайт')
                        # Replace 'lightson' with 'crmsum'
                        if isinstance(sub_value, str) and 'lightson' in sub_value:
                            nested_item[sub_key] = sub_value.replace('lightson', 'crmsum')

        # Handle the main item level as well
        for key in item:
            if isinstance(item[key], str) and 'lightson' in item[key]:
                item[key] = item[key].replace('lightson', 'crmsum')
            
            # Replace email domain with 'Fedor'
            if key == 'custom_fields_values':
                for field in item[key]:
                    if field['field_code'] == 'EMAIL':
                        for value in field['values']:
                            value['value'] = 'Fedor'

        # Replace 'Карты офиса' with 'Воронка продаж'
        if 'pipeline' in item and item['pipeline'] == 'Карты офиса':
            item['pipeline'] = 'Воронка продаж'
            
        # Remove last name from 'responsible_user_id'
        if 'responsible_user_id' in item:
            item['responsible_user_id'] = remove_last_name(item['responsible_user_id'])
            
            # Replace email with 'Федор'
            if item['responsible_user_id'] == 'mlenzovet@gmail.com':
                item['responsible_user_id'] = 'Федор'
            
        # Remove last name from 'responsible_user' -> 'id'
        for key in ['after', 'before']:
            if key in item:
                for nested_item in item[key]:
                    if 'responsible_user' in nested_item:
                        nested_item['responsible_user']['id'] = remove_last_name(nested_item['responsible_user']['id'])
                        
                        # Replace email with 'Федор'
                        if nested_item['responsible_user']['id'] == 'mlenzovet@gmail.com':
                            nested_item['responsible_user']['id'] = 'Федор'
            
    return data_list

def replace_lightson_with_crmsum(data):
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, str):
                data[key] = value.replace('lightson', 'crmsum')
            else:
                replace_lightson_with_crmsum(value)
    elif isinstance(data, list):
        for item in data:
            replace_lightson_with_crmsum(item)
    return data


            
specific_data = replace_lightson_with_crmsum(specific_data)
specific_data = replace_texts_in_data(specific_data)
specific_data

[{'sale': 0,
  'responsible_user_id': 'Валерия',
  'pipeline': 'Воронка продаж',
  'lead_status': 'Получить запрос'},
 {'after': [{'link': {'entity': {'type': 'contact', 'id': 47888395}}}],
  'before': [],
  'contact_info': {'id': 47888395,
   'name': 'Юрий Федосеев',
   'first_name': 'Юрий',
   'last_name': 'Федосеев',
   'responsible_user_id': 2923960,
   'group_id': 415390,
   'created_by': 2923960,
   'updated_by': 2923960,
   'created_at': '2022-09-01 20:20',
   'updated_at': '2022-09-13 13:35',
   'closest_task_at': None,
   'is_deleted': False,
   'is_unsorted': False,
   'custom_fields_values': [{'field_id': 403487,
     'field_name': 'Email',
     'field_code': 'EMAIL',
     'field_type': 'multitext',
     'values': [{'value': 'fedoseevya@hals-development.ru',
       'enum_id': 850537,
       'enum_code': 'WORK'}]}],
   'account_id': 15895585,
   '_links': {'self': {'href': 'https://crmsum.amocrm.ru/api/v4/contacts/47888395?page=1&limit=250'}},
   '_embedded': {'tags': [], 'co

In [24]:
events_final = events_list.drop(['lead_status', 'pipline', 'responsible', 'client', 'company'], axis=1)
events_final['created_at'] = pd.to_datetime(events_final['created_at'], unit='s')
events_final['created_at'] = events_final['created_at'].dt.strftime('%Y-%m-%d %H:%M')
manager_id_to_name = dict(zip(managers_list['id'], managers_list['name']))
events_final['created_by'] = events_final['created_by'].map(manager_id_to_name).fillna(events_final['created_by'])
events_final['created_by'] = [' '.join(str(item).split()[:1]) for item in events_final['created_by'].values]
events_final

,type,entity_id,created_by,created_at,specific_data,sale
0,initial,24651711,Валерия,2022-09-01 20:20,"{'sale': 0, 'responsible_user_id': 'Валерия', 'pipeline': 'Воронка продаж', 'lead_status': 'Получить запрос'}",0.0
1,entity_linked,24651711,Валерия,2022-09-01 20:20,"{'after': [{'link': {'entity': {'type': 'contact', 'id': 47888395}}}], 'before': [], 'contact_info': {'id': 47888395, 'name': 'Юрий Федосеев', 'first_name': 'Юрий', 'last_name': 'Федосеев', 'responsible_user_id': 2923960, 'group_id': 415390, 'created_by': 2923960, 'updated_by': 2923960, 'created_at': '2022-09-01 20:20', 'updated_at': '2022-09-13 13:35', 'closest_task_at': None, 'is_deleted': False, 'is_unsorted': False, 'custom_fields_values': [{'field_id': 403487, 'field_name': 'Email', 'field_code': 'EMAIL', 'field_type': 'multitext', 'values': [{'value': 'fedoseevya@hals-development.ru', 'enum_id': 850537, 'enum_code': 'WORK'}]}], 'account_id': 15895585, '_links': {'self': {'href': 'https://crmsum.amocrm.ru/api/v4/contacts/47888395?page=1&limit=250'}}, '_embedded': {'tags': [], 'companies': []}}}",0.0
2,lead_added,24651711,Валерия,2022-09-01 20:20,"{'after': [], 'before': []}",0.0
3,name_field_changed,24651711,Валерия,2022-09-01 20:20,"{'after': [{'name_field_value': {'name': 'ГАЛС'}}], 'before': []}",0.0
4,custom_field_487035_value_changed,24651711,Ева,2022-09-05 09:10,"{'after': [{'custom_field_value': {'field_id': 'Почему не купили', 'field_type': 4, 'enum_id': 955727, 'text': '16 Ждет от нас'}}], 'before': []}",0.0
5,note,24651711,Валерия,2022-09-15 09:14,"{'text': 'много ебли в тг :)', 'note_type': 'common', 'responsible_user_id': 'Валерия', 'updated_at': '2022-09-15 09:14'}",0.0
6,common_note_added,24651711,Валерия,2022-09-15 09:14,"{'after': [{'note': {'id': 250270209}}], 'before': []}",0.0
7,custom_field_487035_value_changed,24651711,Софья,2022-09-15 09:20,"{'after': [{'custom_field_value': {'field_id': 'Почему не купили', 'field_type': 4, 'enum_id': 955373, 'text': '5 Тестируют систему'}}], 'before': [{'custom_field_value': {'field_id': 'Почему не купили', 'field_type': 4, 'enum_id': 955727, 'text': '16 Ждет от нас'}}]}",0.0
8,entity_direct_message,24651711,Софья,2022-10-20 09:36,"{'after': [], 'before': []}",0.0
9,entity_direct_message,24651711,Валерия,2022-10-20 09:38,"{'after': [{'message': {'id': 'df72796b-13ac-469e-b216-aaeb43503a0d'}}], 'before': []}",0.0


In [26]:
events_final.to_csv('events_good.csv')